<a href="https://colab.research.google.com/github/agusnieto77/taller_pict_2019/blob/main/Colabs/ACEP_funciones_desarrollo_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 3

In [4]:
devtools::install_github("agusnieto77/ACEP")

sys          (3.4.1  -> 3.4.2 ) [CRAN]
jsonlite     (1.8.3  -> 1.8.4 ) [CRAN]
png          (NA     -> 0.1-8 ) [CRAN]
here         (NA     -> 1.0.1 ) [CRAN]
RcppTOML     (NA     -> 0.2.2 ) [CRAN]
Rcpp         (NA     -> 1.0.10) [CRAN]
reticulate   (NA     -> 1.28  ) [CRAN]
tokenbrowser (NA     -> 0.1.5 ) [CRAN]
digest       (0.6.30 -> 0.6.31) [CRAN]
igraph       (NA     -> 1.4.3 ) [CRAN]
tidygeocoder (NA     -> 1.0.5 ) [CRAN]
udpipe       (NA     -> 0.8.11) [CRAN]
spacyr       (NA     -> 1.2.1 ) [CRAN]
rsyntax      (NA     -> 0.1.4 ) [CRAN]


Installing 14 packages: sys, jsonlite, png, here, RcppTOML, Rcpp, reticulate, tokenbrowser, digest, igraph, tidygeocoder, udpipe, spacyr, rsyntax

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpW5d7HT/remotes2d33ce5ec81/agusnieto77-ACEP-f401cfe/DESCRIPTION’ ... OK
* preparing ‘ACEP’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘ACEP_0.0.3.9001.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [21]:
require(ACEP)

In [22]:
acep_context("La UTA inició un paro de colectivos por tiempo indeterminado", "paro")

doc_id,contexto
<int>,<chr>
1,un | paro | de


In [23]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "paro")

doc_id,contexto
<int>,<chr>
1,inicio | paro | colectivos


In [24]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "paro", izq = 2, der = 2)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | colectivos tiempo


In [25]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "paro", izq = 2, der = 3)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | colectivos tiempo indeterminado


In [26]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "inicio paro colectivos", izq = 1, der = 3)

doc_id,contexto
<int>,<chr>
1,uta | inicio paro colectivos | tiempo indeterminado


In [27]:
texto <-     
  c("La UTA inició un paro de colectivos por tiempo indeterminado.", 
    "El gremio que nuclea a los choferes lo resolvió por un atraso salarial.",
    "La medida de fuerza comenzó a partir de las 18.",
    "Por un atraso salarial, la Unión Tranviarios Automotor (UTA) inició 
    un paro por tiempo indeterminado a partir de las 18 de este lunes, 
    según confirmaron fuentes del gremio que nuclea a los choferes.")

In [28]:
acep_context(acep_clean(texto), "paro", izq = 2, der = 2)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | colectivos tiempo
4,uta inicio | paro | tiempo indeterminado


In [29]:
acep_context(acep_clean(texto), c("paro", "medida fuerza"), izq = 2, der = 2)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | colectivos tiempo
4,uta inicio | paro | tiempo indeterminado
3,| medida fuerza comenzo partir | comenzo partir


In [30]:
texto2 <-     
  c("La UTA inició un paro por tiempo indeterminado. El paro de la UTA no tiene relación con el paro de Moyano. 
    El nuevo paro de colectivos es total.", 
    "El gremio que nuclea a los choferes resolvió el paro por un atraso salarial. El paro es total.",
    "La medida de fuerza comenzó a partir de las 18. El gremio que nuclea a los choferes resolvió el paro por un atraso salarial.",
    "Por un atraso salarial, la Unión Tranviarios Automotor (UTA) inició un paro por tiempo indeterminado. A partir de las 18 de este lunes, 
    según confirmaron fuentes del gremio que nuclea a los choferes que están en paro por tiempo indeterminado.")

In [31]:
acep_context(acep_clean(texto2), c("paro", "medida fuerza"), izq = 4, der = 4)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | tiempo indeterminado | paro | uta
1,relacion | paro | moyano nuevo | paro | colectivos
2,gremio nuclea choferes resolvio | paro | atraso salarial | paro | total
3,gremio nuclea choferes resolvio | paro | atraso salarial
4,tranviarios automotor uta inicio | paro | tiempo indeterminado partir segun
4,gremio nuclea choferes estan | paro | tiempo indeterminado
3,| medida fuerza comenzo partir gremio nuclea choferes resolvio paro atraso salarial | comenzo partir gremio nuclea


### Posible solución

In [32]:
acep_context <- function(texto, clave, izq = 1, der = 1){
  nwi <- "[[:graph:]]*[[:space:]]*"
  nwd <- "[[:space:]]*[[:graph:]]*"
  lista_frases <- c()
  
  for (c in clave) {
    claveb <- paste0(c, "[[:graph:]]*", collapse = "|")
    capturar <- paste0("(", paste0(rep(nwi,izq), collapse = ""),
                       claveb, paste0(rep(nwd, der), collapse = ""), ")")
    
    for (o in seq_along(texto)) {
      oraciones <- unlist(strsplit(texto[o], "(?<=[a-z]\\.|\\?|\\!)\\s*(?=[A-Z]|\n[A-Z])", perl=T))
      
      for (i in seq_along(oraciones)) {
        vector <- unlist(regmatches(oraciones[i], gregexpr(capturar, oraciones[i])))
        
        for (v in seq_along(vector)) {
          claves <- unlist(regmatches(vector[v], gregexpr(claveb, vector[v])))
          lista <- sapply(seq_along(vector[v]), function(x) sub(claveb, paste0("\\| ", claves[x], " \\|"), vector[v][x]))
          contexto <- trimws(strsplit(lista, "|", fixed = TRUE)[[1]])
          lista_frases <- rbind(lista_frases, data.frame(doc_id = o, oraciones_id = i, texto = lista, w_izq = contexto[1], key = contexto[2], w_der = contexto[3]))
        }
      }
    }
  }

  return(lista_frases)
}

In [33]:
acep_context(acep_clean(texto2), c("paro", "medida fuerza"), izq = 4, der = 4)

doc_id,oraciones_id,texto,w_izq,key,w_der
<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,1,uta inicio | paro | tiempo indeterminado paro uta,uta inicio,paro,tiempo indeterminado paro uta
1,1,relacion | paro | moyano nuevo paro colectivos,relacion,paro,moyano nuevo paro colectivos
2,1,gremio nuclea choferes resolvio | paro | atraso salarial paro total,gremio nuclea choferes resolvio,paro,atraso salarial paro total
3,1,gremio nuclea choferes resolvio | paro | atraso salarial,gremio nuclea choferes resolvio,paro,atraso salarial
4,1,tranviarios automotor uta inicio | paro | tiempo indeterminado partir segun,tranviarios automotor uta inicio,paro,tiempo indeterminado partir segun
4,1,gremio nuclea choferes estan | paro | tiempo indeterminado,gremio nuclea choferes estan,paro,tiempo indeterminado
3,1,| medida fuerza | comenzo partir gremio nuclea,,medida fuerza,comenzo partir gremio nuclea


In [34]:
acep_context(texto2, c("paro", "medida de fuerza"), izq = 4, der = 4)

doc_id,oraciones_id,texto,w_izq,key,w_der
<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,1,La UTA inició un | paro | por tiempo indeterminado.,La UTA inició un,paro,por tiempo indeterminado.
1,2,El | paro | de la UTA no,El,paro,de la UTA no
1,2,tiene relación con el | paro | de Moyano.,tiene relación con el,paro,de Moyano.
1,3,El nuevo | paro | de colectivos es total.,El nuevo,paro,de colectivos es total.
2,1,los choferes resolvió el | paro | por un atraso salarial.,los choferes resolvió el,paro,por un atraso salarial.
2,2,El | paro | es total.,El,paro,es total.
3,1,los choferes resolvió el | paro | por un atraso salarial.,los choferes resolvió el,paro,por un atraso salarial.
4,1,Automotor (UTA) inició un | paro | por tiempo indeterminado.,Automotor (UTA) inició un,paro,por tiempo indeterminado.
4,2,choferes que están en | paro | por tiempo indeterminado.,choferes que están en,paro,por tiempo indeterminado.


In [35]:
acep_context(acep_clean(texto2), c("paro", "medida fuerza"), izq = 4, der = 4)

doc_id,oraciones_id,texto,w_izq,key,w_der
<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,1,uta inicio | paro | tiempo indeterminado paro uta,uta inicio,paro,tiempo indeterminado paro uta
1,1,relacion | paro | moyano nuevo paro colectivos,relacion,paro,moyano nuevo paro colectivos
2,1,gremio nuclea choferes resolvio | paro | atraso salarial paro total,gremio nuclea choferes resolvio,paro,atraso salarial paro total
3,1,gremio nuclea choferes resolvio | paro | atraso salarial,gremio nuclea choferes resolvio,paro,atraso salarial
4,1,tranviarios automotor uta inicio | paro | tiempo indeterminado partir segun,tranviarios automotor uta inicio,paro,tiempo indeterminado partir segun
4,1,gremio nuclea choferes estan | paro | tiempo indeterminado,gremio nuclea choferes estan,paro,tiempo indeterminado
3,1,| medida fuerza | comenzo partir gremio nuclea,,medida fuerza,comenzo partir gremio nuclea
